In [10]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, CustomJS, Select,MultiSelect
from bokeh.models.tools import HoverTool
from bokeh.transform import factor_cmap, cumsum
from bokeh.layouts import row, column, gridplot
from bokeh.models import DatetimeTickFormatter
from bokeh.models import NumeralTickFormatter


In [11]:
#load data
process_dataset = pd.read_csv('archive/supplier_stock_value_projected_FG.csv')
process_dataset['Date'] = pd.to_datetime(process_dataset['Date'],format='%d-%m-%Y')
process_dataset = process_dataset.sort_values('Date',ascending=True) 

##### plot by item
#link between pd and bokeh for original data
src1 = ColumnDataSource(process_dataset)


# create CDS for empty dataframe
filteredSource = ColumnDataSource(data=dict(Date=[],Stock_value=[]))


p = figure(title="Invetory level Finished Goods",
                x_axis_type="datetime",
                x_axis_label='Date',
                y_axis_label='Stock_value')


item_draw =p.line(x='Date',y='Stock_value',source = filteredSource,legend_label="Stock level",hover_color = 'red')

#format y axis
p.left[0].formatter.use_scientific = False
p.yaxis[0].formatter = NumeralTickFormatter(format="$0,000")

#format x axis
p.xaxis.formatter=DatetimeTickFormatter(
        days=["%d/%m/%Y"],
    )


#add hovertool
p.add_tools(HoverTool(
    tooltips=[
        ( "Date","$x{%F}"),
        ( "Stock_value", "$y{"+f"0.00"+" a}" )
    ],
    formatters={
        '$x' : 'datetime',
    },
))

p.legend.location = 'bottom_right'

vendor_list = sorted(process_dataset['Vendor'].unique().tolist())

#get list of item for box
item_list= ['Please choose item...'] + vendor_list


select = Select(title='Vendors', value=item_list[0], options=item_list)

callback = CustomJS(
    args=dict(src1=src1,filteredSource=filteredSource,select=select),
    code= """
    const data = src1.data;
    const item_filtered = select.value;
    const plotdata = filteredSource.data;
    plotdata['Stock_value']=[];
    plotdata['Date']=[];
    
    for (var i=0; i<data['Vendor'].length; i++) {
        if(data['Vendor'][i] == item_filtered ) {
            plotdata['Stock_value'].push(data['Stock_value'][i])
            plotdata['Date'].push(data['Date'][i])
        }
    }
    filteredSource.change.emit();
    """
)
select.js_on_change('value',callback)


layout = row(p,column(select))
show(layout)

In [12]:
process_dataset

,Vendor,Date,Stock_value
0,0 - 0,2022-06-15,0.000
255,70508 - Liko Hardware Company Limited,2022-06-15,0.000
320,71551 - ASSA ABLOY Electronic Security Hardware,2022-06-15,3852.730
210,50005 - ASSA INDUSTRIES,2022-06-15,20.700
323,71622 - Integrated Micro-Electronic Inc,2022-06-15,13495.540
...,...,...,...
3663,30284 - Austral Wright Metals,2022-12-28,-338942.800
1531,37326 - PREGEX (H.K.) COMPANY LIMITED,2022-12-28,15291.865
5873,70676 - ASSA ABLOY Japan Osaka,2022-12-28,0.000
7225,71815 - Waughs Industrial Supplies P/L,2022-12-28,-491.700
